In [47]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import altair as alt
import panel as pn
from function import *

# **PlotResults**

## Read the CSV

In [48]:
# name of the CSV
n_file = 'GOA_Baseline.csv'      # file name
location = 'Output_Data'               # file address 
n_file_r = pd.read_csv(location +'/' + n_file, low_memory=False)

## Name of techs

Read the excel file that contains the name of technologies and the alias or easy names

In [86]:
#names of tech  The file Tech_names contains a dictionary with the techs and easy names for the Goa model

#names of tech elecetrcity
elec_en = pd.read_excel('Tech_names_Goa.xlsx', sheet_name='Power', usecols='A,B,C,D')
#names of other sectors
fe_en = pd.read_excel('Tech_names_Goa.xlsx', sheet_name='sectoral_mix', usecols='A,D,E,F')
#tn names of all tech joined for emissions
emi_en = pd.concat([elec_en,fe_en], ignore_index=True)
# alias for primary names
pe_en = pd.read_excel('Tech_names_Goa.xlsx', sheet_name='primary_e', usecols='A,B,C')

C:\Users\shank\anaconda3\envs\message1.1\lib\site-packages\ipykernel_launcher.py:8: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [87]:
#sectorial mix
sector_names = pd.read_excel('Tech_names_Goa.xlsx', usecols='A, B,C,D,E,F', sheet_name='sectoral_mix')

sector_namesT = sector_names[sector_names['Sector'].isin(['Public','Private','Freight'])] # to use in rate of use by tech for the transport
sector_namesOTH = sector_names[~sector_names['Sector'].isin(['Public','Private','Freight'])]

In [88]:
colors = pd.concat([sector_names,pe_en,emi_en])[['Tech_name','Sector','Fuel','Color']]
fuel_colors = colors[['Fuel','Color']].drop_duplicates().set_index('Fuel')
sec_colors = colors[['Sector','Color']].drop_duplicates().set_index('Sector')
tec_colors = colors[['Tech_name','Color']].drop_duplicates().set_index('Tech_name')

fuel_colors = fuel_colors[~fuel_colors.index.duplicated()].to_dict()['Color']
sec_colors = sec_colors[~sec_colors.index.duplicated(keep='first')].to_dict()['Color']
tec_colors = tec_colors[~tec_colors.index.duplicated(keep='first')].to_dict()['Color']

C:\Users\shank\anaconda3\envs\message1.1\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




## Reading csv file and segregating

In [89]:
act = n_file_r.loc[n_file_r.NAME=='ProdByTech']
pe = n_file_r.loc[n_file_r.NAME=='ProdAnn']
new_cap = n_file_r.loc[n_file_r.NAME=='NewCapacity']
tot_cap = n_file_r.loc[n_file_r.NAME=='TotCapacityAnn']
dem = n_file_r.loc[n_file_r.NAME=='RateOfUseByTech']
aet = n_file_r.loc[n_file_r.NAME=='AnnTechEmission']
inv = n_file_r.loc[n_file_r.NAME=='DiscCapitalInvestment']

# Primary Energy

In [90]:
sply = act.copy()
sply = (pd.merge(sply, pe_en[['TECHNOLOGY', 'primary_names','Color']] , on='TECHNOLOGY'))
sply = sply.groupby(['YEAR','primary_names','Color'], as_index=False).sum()
sply = sply[sply['VALUE'] > 0]
base = alt.Chart(sply.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['primary_names'],bind='legend')
cmap = sply[['primary_names','Color']].drop_duplicates()
grouped = base.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Primary Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('primary_names', scale=alt.Scale(domain=cmap['primary_names'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['primary_names','YEAR','sum(VALUE)']
            ).add_selection(selection).interactive()
text = base.mark_text(dx=0,dy=-40, color='black').encode(
    x=alt.X('YEAR:N', stack='zero'),
    y=alt.Y('sum(VALUE)'),
    text=alt.Text('sum(VALUE)', format='.2f')
)

# Each sector plot filtered by clicking the stacked plot
one_group = grouped.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Primary Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('primary_names', scale=alt.Scale(domain=cmap['primary_names'].values.tolist(), range=cmap['Color'].values.tolist())),
    tooltip=['primary_names','sum(VALUE)','YEAR'],
    row='primary_names',
).transform_filter( 
    selection
).interactive()


In [91]:
# (grouped)&one_group

In [92]:
# base.mark_line(dx=0,dy=-40, color='black').encode(
#     x=alt.X('YEAR:N', stack='zero'),
#     y=alt.Y('sum(VALUE)',title='Primary Energy in PJ'),
#     tooltip=['sum(VALUE)','YEAR'],
# )

# Final Energy by Fuel and Sectors

In [93]:
trp = dem.loc[dem['TECHNOLOGY'].isin(np.array(sector_namesT['TECHNOLOGY']))]
trp = rate_us_ts(trp)
all_fe = act.copy().append(trp, ignore_index=True)
final_energy_all = (pd.merge(all_fe, sector_names[['TECHNOLOGY', 'Tech_name','Sector','Fuel','Color','Sector_Color']] , on='TECHNOLOGY'))

final_energy_fuel = final_energy_all.groupby(['YEAR','Fuel','Color'], as_index=False).sum()
final_energy_sec = final_energy_all.groupby(['YEAR','Sector','Sector_Color'], as_index=False).sum()

final_energy_fuel = final_energy_fuel[final_energy_fuel['VALUE'] > 0]
final_energy_sec = final_energy_sec[final_energy_sec['VALUE'] > 0]

fe_fuel = alt.Chart(final_energy_fuel.dropna(),width=614,height=437)
fe_sec = alt.Chart(final_energy_sec.dropna(),width=614,height=437)

selection = alt.selection_multi(fields=['Fuel'],bind='legend')
cmap = final_energy_fuel[['Fuel','Color']].drop_duplicates()

fe_by_fuel = fe_fuel.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Fuel', scale=alt.Scale(domain=cmap['Fuel'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['Fuel','YEAR','sum(VALUE)']
            ).add_selection(selection).interactive()
cmap = final_energy_sec[['Sector','Sector_Color']].drop_duplicates()
fe_by_sec = fe_sec.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['Sector','YEAR','sum(VALUE)']
            ).add_selection(selection).interactive()


text = fe_fuel.mark_text(dx=0,dy=-40, color='black').encode(
    x=alt.X('YEAR:N', stack='zero'),
    y=alt.Y('sum(VALUE)'),
    text=alt.Text('sum(VALUE)', format='.2f')
)



# Each sector plot filtered by clicking the stacked plot
one_fuel = fe_by_fuel.mark_area().encode(
    alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Fuel'), 
    tooltip=['Fuel','sum(VALUE)','YEAR'],
    row='Fuel',
).transform_filter( 
    selection
).interactive()

one_sec = fe_by_sec.mark_area().encode(
    alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Sector'), 
    tooltip=['Sector','sum(VALUE)','YEAR'],
    row='Sector',
).transform_filter( 
    selection
).interactive()

In [94]:
# fe_by_fuel&one_fuel
# fe_by_sec&one_sec

## Emissions

In [107]:
#Emissions

#annual emission by tech
elec_emi = (pd.merge(aet.copy(), elec_en, on='TECHNOLOGY'))
elec_emi['Sector'] = 'Power'
elec_emi.rename({'Color': 'Sector_Color'}, axis=1,inplace=True)
fe_emi = (pd.merge(aet.copy(), fe_en, on='TECHNOLOGY'))
fe_emi.drop(columns=['Color'],inplace=True)
fe_emi["Sector"] = [sector[0] for sector in fe_emi['Tech_name'].str.split('-')]
all_emi = pd.concat([elec_emi[["Tech_name",'Sector','YEAR','Sector_Color','VALUE']],fe_emi[["Tech_name",'Sector','YEAR','Sector_Color','VALUE']]])

In [108]:
emissions = alt.Chart(all_emi.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Sector'],bind='legend')
tot_emi_by_year = emissions.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Emissions in TTCO2e',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Sector'),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Sector','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_emi = tot_emi_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Emissions in TTCO2e',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Sector'), 
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Sector',
).transform_filter( 
    selection
).interactive()

In [110]:
# tot_emi_by_year&one_emi

# COST

## CAP INVESTMENT BY SECTOR

In [14]:
trp_corr = pd.read_excel('Transport_correction.xlsx').set_index('Tech_name').to_dict()

In [15]:
inv

,NAME,VALUE,SCENARIO,REGION,REGION2,DAYTYPE,FUEL,EMISSION,DAILYTIMEBRACKET,SEASON,TIMESLICE,MODE_OF_OPERATION,STORAGE,TECHNOLOGY,YEAR
16507,DiscCapitalInvestment,7.588819e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2019.0
16508,DiscCapitalInvestment,7.439289e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2021.0
16509,DiscCapitalInvestment,5.524225e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2022.0
16510,DiscCapitalInvestment,5.469529e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2023.0
16511,DiscCapitalInvestment,7.220501e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2024.0
16512,DiscCapitalInvestment,5.361758e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2025.0
16513,DiscCapitalInvestment,7.078228e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2026.0
16514,DiscCapitalInvestment,7.008147e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2027.0
16515,DiscCapitalInvestment,5.204070e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2028.0
16516,DiscCapitalInvestment,6.870059e-01,0,GOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEMRESBGSCKG,2029.0


In [23]:
new_inv = inv.copy()
new_inv = (pd.merge(new_inv, emi_en, on='TECHNOLOGY'))
new_inv.drop(new_inv.iloc[:, 2:13], axis=1, inplace=True)
for key,value in trp_corr['km'].items():
    new_inv.loc[new_inv['Tech_name'].str.contains(key+'-'), 'VALUE'] /= value


,NAME,VALUE,TECHNOLOGY,YEAR,Tech_name
0,DiscCapitalInvestment,0.758882,DEMRESBGSCKG,2019.0,Cooking urban-Biogas
1,DiscCapitalInvestment,0.743929,DEMRESBGSCKG,2021.0,Cooking urban-Biogas
2,DiscCapitalInvestment,0.552422,DEMRESBGSCKG,2022.0,Cooking urban-Biogas
3,DiscCapitalInvestment,0.546953,DEMRESBGSCKG,2023.0,Cooking urban-Biogas
4,DiscCapitalInvestment,0.722050,DEMRESBGSCKG,2024.0,Cooking urban-Biogas
5,DiscCapitalInvestment,0.536176,DEMRESBGSCKG,2025.0,Cooking urban-Biogas
6,DiscCapitalInvestment,0.707823,DEMRESBGSCKG,2026.0,Cooking urban-Biogas
7,DiscCapitalInvestment,0.700815,DEMRESBGSCKG,2027.0,Cooking urban-Biogas
8,DiscCapitalInvestment,0.520407,DEMRESBGSCKG,2028.0,Cooking urban-Biogas
9,DiscCapitalInvestment,0.687006,DEMRESBGSCKG,2029.0,Cooking urban-Biogas


In [24]:
new_inv = inv.copy()
new_inv = (pd.merge(new_inv, emi_en, on='TECHNOLOGY'))
new_inv.drop(new_inv.iloc[:, 2:13], axis=1, inplace=True)
for key,value in trp_corr['km'].items():
    new_inv.loc[new_inv['Tech_name'].str.contains(key+'-'), 'VALUE'] /= value
new_inv.drop(columns=['Electr'], inplace=True)
new_inv["Sector"] = [sector[0] for sector in new_inv['Tech_name'].str.split('-')]
new_inv

,NAME,VALUE,TECHNOLOGY,YEAR,Tech_name,Sector
0,DiscCapitalInvestment,0.758882,DEMRESBGSCKG,2019.0,Cooking urban-Biogas,Cooking urban
1,DiscCapitalInvestment,0.743929,DEMRESBGSCKG,2021.0,Cooking urban-Biogas,Cooking urban
2,DiscCapitalInvestment,0.552422,DEMRESBGSCKG,2022.0,Cooking urban-Biogas,Cooking urban
3,DiscCapitalInvestment,0.546953,DEMRESBGSCKG,2023.0,Cooking urban-Biogas,Cooking urban
4,DiscCapitalInvestment,0.722050,DEMRESBGSCKG,2024.0,Cooking urban-Biogas,Cooking urban
5,DiscCapitalInvestment,0.536176,DEMRESBGSCKG,2025.0,Cooking urban-Biogas,Cooking urban
6,DiscCapitalInvestment,0.707823,DEMRESBGSCKG,2026.0,Cooking urban-Biogas,Cooking urban
7,DiscCapitalInvestment,0.700815,DEMRESBGSCKG,2027.0,Cooking urban-Biogas,Cooking urban
8,DiscCapitalInvestment,0.520407,DEMRESBGSCKG,2028.0,Cooking urban-Biogas,Cooking urban
9,DiscCapitalInvestment,0.687006,DEMRESBGSCKG,2029.0,Cooking urban-Biogas,Cooking urban


In [ ]:
capinv = alt.Chart(new_inv.dropna(),width=1228,height=437)
selection = alt.selection_multi(fields=['Sector'],bind='legend')
tot_inv_by_year = capinv.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='New Investments in INR Crore',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Sector'),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Sector','sum(VALUE)']
            ).add_selection(selection).interactive()

# Each sector plot filtered by clicking the stacked plot
one_inv = tot_inv_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='New Investments in INR Crore',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.X("Tech_name",title='Technology',axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
    alt.Color('Sector'), 
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Sector',
).transform_filter( 
    selection
).interactive()

In [ ]:
# tot_inv_by_year & one_inv

# Sector Mix

# Electricity Generation

In [ ]:
#Production by tech electrcity

p_tech = act.copy()
#only for the electricity sector
p_tech = p_tech.loc[p_tech['FUEL'].isin(np.array(elec_en['Electr']))]
p_tech = (pd.merge(p_tech, elec_en[['TECHNOLOGY', 'Tech_name']] , on='TECHNOLOGY'))
p_tech = p_tech.groupby(['YEAR','Tech_name', 'TIMESLICE'], as_index=False).sum() 
p_tech.drop(p_tech.columns[4:], axis=1, inplace=True)


In [ ]:
generation = alt.Chart(p_tech.dropna(),width=1228,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
generation_by_year = generation.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name'),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_year = generation_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.X("TIMESLICE:N",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Tech_name'), 
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='YEAR',
).transform_filter( 
    selection
).interactive()


In [ ]:
# generation_by_year&one_year

## Total Capacity

In [ ]:

tot_cap = tot_cap.loc[tot_cap['TECHNOLOGY'].isin(np.array(elec_en['TECHNOLOGY']))]
tot_cap = (pd.merge(tot_cap, elec_en[['TECHNOLOGY', 'Tech_name']] , on='TECHNOLOGY'))
tot_cap = tot_cap.groupby(['YEAR','Tech_name'], as_index=False).sum()
tot_cap.drop(tot_cap[tot_cap.Tech_name == 'IEX-Imports'].index, inplace=True)
tot_cap.drop(tot_cap[tot_cap.Tech_name == 'RE-Imports'].index, inplace=True)
tot_cap.drop(tot_cap.columns[3:], axis=1, inplace=True)


In [ ]:

totcap = alt.Chart(tot_cap.dropna(),width=1228,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
tot_cap_by_year = totcap.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name'),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_cap = tot_cap_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Tech_name'), 
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Tech_name',
).transform_filter( 
    selection
).interactive()

In [ ]:
# tot_cap_by_year&one_cap

## New Capacity

In [ ]:
# New capacity

ncap = new_cap.copy()
ncap = ncap.loc[ncap['TECHNOLOGY'].isin(np.array(elec_en['TECHNOLOGY']))]
ncap = (pd.merge(ncap, elec_en[['TECHNOLOGY', 'Tech_name']] , on='TECHNOLOGY'))
ncap = ncap.groupby(['YEAR','Tech_name'], as_index=False).sum()
ncap.drop(ncap[ncap.Tech_name == 'IEX-Imports'].index, inplace=True)
ncap.drop(ncap[ncap.Tech_name == 'RE-Imports'].index, inplace=True)
ncap.drop(ncap.columns[3:], axis=1, inplace=True)
# acc_ncap.head(5)

In [ ]:
newcap = alt.Chart(ncap.dropna(),width=1228,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
new_cap_by_year = newcap.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name'),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_cap = new_cap_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Tech_name'), 
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Tech_name',
).transform_filter( 
    selection
).interactive()

In [ ]:
# new_cap_by_year&one_cap

In [ ]:
sector = final_energy_all.Sector.drop_duplicates().tolist()[0]
fe_dem = final_energy_all.loc[final_energy_all.Sector==sector,['YEAR','Tech_name','VALUE']]
sec_chart = alt.Chart(fe_dem.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
sec_chart.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name'),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()


In [ ]:
import panel as pn
sectors = final_energy_all.Sector.drop_duplicates().tolist()
def fe_chart(sector):
    fe_dem = final_energy_all.loc[final_energy_all.Sector==sector,['YEAR','Tech_name','VALUE']]
    sec_chart = alt.Chart(fe_dem.dropna(),width=614,height=437)
    chart = sec_chart.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name'),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            )
    return pn.pane.Vega(chart)
     
pn.interact(fe_chart,sector=sectors)

In [ ]:
sectors